<a href="https://colab.research.google.com/github/Sifatkhan-1915020/deeplearning-/blob/main/Customer_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bitext-gen-ai-chatbot-customer-support-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4624298%2F7879155%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T100250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1b4ebdf36c470eaffbeabbbb98c3ee1c49aa2b804ac53afcaf3ee61269ebc348e780a15c796706c313b9361f1ca114a39502fcf976aee4423ade455614b4febbad60e0cebb3eb1d0d2b944803de2dee3e993d072d8969c56ff124e803f226858dc87076e58f13bebb6874ea3154785de1f685a94b20e92191ec242bc0f9c7b8e5c6f00c911f0a4d03438f0196f9b48324b5ca792d0e4f0cc30dbfaf17f712cbceff12bdbe97ffc5e1fec4a6463cbec1ffa2ecc91f1330253e96953bd875b2c9c9044392d9f6ebae8e5b3f59a6954db9dc1a1c4bd39a0eaa645f201aa9cf2d6643d344ca11590175ffab3574d7692e099f9e3bd8702da6069f6850d029673ab87,faq-chatbot:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5211420%2F8690993%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T100250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8223944d0c8c0aec7cf2e3eeb1998744914f27d284e23d64d2b907ab261dece5d62c6dedf1dd260e7ddaf74f0e350b48d2c9607ecfeca8abf6eba365c0827cef8f207059054419e93c59ced82f3e89db57d95f4d35e12182b25f48c761c9500e0970b4bb44711a36c00035013fc63bb0abf91dbe059c5c28e74c7205a470336cfcc83adc855ccde3b1371ced2374e34b3305d49677d93d52906da2762231679fb89f43f16e3942b62a14b39e2f84688c3e774ae1ac4d9ef416a0d05b4707933099b3dc7c007e516fe2c6f7959db8c83385a777468829d7d3c83321e8a5fc1f73bf937045b1dd3342c16b42f0a8c877c0cc08ddfc3913c579ffb9737a19e386c6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!cd kaggle
!pip install --upgrade pip
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [ ]:
import os
import keras
import keras_nlp
import pandas as pd
import numpy as np
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/bitext-gen-ai-chatbot-customer-support-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')
df = df[['instruction','response']]
df.head()
df.tail()

In [ ]:
gemma = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

In [ ]:
# gemma_1 = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

In [ ]:
gemma.summary()


In [ ]:
template = "Instruction:\n{instruction}\n\n\nResponse:\n{response}"
filename = "/kaggle/input/bitext-gen-ai-chatbot-customer-support-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"

In [ ]:
prompt = template.format(
    instruction="question about cancelling order  ",
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction=" i have a question about cancelling oorder.",
    intent = "",
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction="i need help cancelling puchase .",
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
# prompt = template.format(
#     instruction="give me some slice of life anime.",
#     response="",
# )
# print(gemma.generate(prompt, max_length=256))

In [ ]:
# prompt = template.format(
#     instruction="give me some summary on hajime on ippo.",
#     response="",
# )
# print(gemma.generate(prompt, max_length=256))

In [ ]:
for i,j in df.iterrows():
    j['instruction'],j['response'] = j['instruction'].replace("{{Order Number}}", ""),j['response'].replace("{{Order Number}}", "").replace("{{","").replace("}}","")
# df = df[df['category'] != 'SHIPPING']
# new_df  = pd.DataFrame()
# count = 0
# for j in df['category'].unique():
#     a = df[df['category'] == j]
#     if count == 2:
#         break
#     for i in a['intent'].unique():
#         temp_df = df.loc[df['intent'] == i].head(400)
#         new_df = pd.concat([new_df, temp_df],ignore_index = True)
#     count += 1

# df = new_df

In [ ]:
df.tail()

In [ ]:
df_2 = pd.read_json("/kaggle/input/faq-chatbot/archive/Ecommerce_FAQ_Chatbot_dataset.json")
df_3 = pd.read_csv("/kaggle/input/faq-chatbot/archive (1)/faq.csv")
df_4 = pd.read_csv("/kaggle/input/faq-chatbot/archive (2)/fact-base-tesco.csv")
df_4 = df_4[["Question",'Answer']]

In [ ]:
df_3 = df_3.rename(columns={'Questions': 'instruction','Answers':'response'})
df_4 = df_4.rename(columns={'Question': 'instruction','Answer':'response'})

In [ ]:
f = []
for i,row in df_2.iterrows():
    for column in df_2.columns:
        data = row[column]
        f.append({"instruction": data['question'], "response": data['answer']})
df_2 = pd.DataFrame(f)

In [ ]:
df_2.head(),df_3.head(),df_4.head()

In [ ]:
# for i,j in df.iterrows():
#     j['instruction'],j['response'] = j['instruction'].replace("{{Order Number}}", ""),j['response'].replace("{{Order Number}}", "").replace("{{","").replace("}}","")
# df = df[df['category'] != 'SHIPPING']
# new2_df  = pd.DataFrame()
# count = 0
# for j in df['category'].unique():
#     a = df[df['category'] == j]
#     if count == 2:
#         break
#     for i in a['intent'].unique():
#         temp_df = df.loc[df['intent'] == i].tail(400)
#         new2_df = pd.concat([new2_df, temp_df],ignore_index = True)
#     count += 1

# df2 = new2_df

In [ ]:
# df2.tail()

In [ ]:
print(type(df_2)
,type(df_3)
,type(df_4))

In [ ]:
df = pd.concat([df, df_2, df_3, df_4], ignore_index=True)

In [ ]:
df.tail()

In [ ]:
#Remove null values
df.dropna(inplace=True)
df.isnull().any()

In [ ]:
# Convert dataframe to a list with instruction and response
rows = []
for index, row in df.iterrows():
    rows.append(f"Instruction:\t{row['instruction']}\n\nResponse:\n{row['response']}.")


In [ ]:
print(rows[-1])

In [ ]:
gemma.backbone.enable_lora(rank=4)

In [ ]:
gemma.preprocessor.sequence_length = 400

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma.fit(rows, epochs=1, batch_size=1)

In [ ]:
prompt = template.format(
    instruction="question about cancelling order  ",
    response=""
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
import random

In [ ]:

prompt = template.format(
    instruction = df.iloc[random.randint(0,2000)]['instruction'],
    response="",
)
print(gemma.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    instruction="i need help cancelling puchase",
    response="",
)
response = gemma.generate(prompt, max_length=512)
print(response)

In [ ]:
# Assuming 'gemma' is your Gemma model
model_path = "/kaggle/working/model.h5"  # Adding a valid '.h5' extension

# Save the model
gemma.save(model_path)

In [ ]:
# prompt: attach drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
prompt = template.format(
    instruction="i need help cancelling a purchase. Please provide detailed steps on how to proceed.",
    response="",
)

print(gemma.generate(prompt, max_length=256))
